# Welcome!
If you have made it this far, congratulations, I am kind of shocked this reproducibility thing worked.

First, we will extract the experimental info from `allsentences.txt`. Sentences, as well as their item and condition numbers. This is someone else's magic code I am borrowing, so I hope I return it eventually!

In [1]:
script_file = open("ad.script", 'r')		# Script file, output from eyetrack_reading.py

#the Item number of the first item, as counted in the EyeTrack .script file
#(Item number comes after the "I" in the trial id number)

first_item = 1    
    
#the number of items

num_items = 180

#the Condition number of the first item, as counted in the EyeTrack .script file
#(Condition number comes after the "E" in the trial id number)

first_cond = 1

#the total number of conditions, as counted in the EyeTrack .script file
#(If multiple experiments with different numbers of conditions were included in the
#script, then the number of conditions will be a multiple of the actual number.)

num_conds = 5

#define range of item numbers:
items = range(first_item, first_item+num_items)
conds = range(first_cond, first_cond+num_conds)

#make list of lines in script_file:
lines = script_file.readlines()
script_file.close()

#define empty list for sentences:
sentences = []
#define variable that keeps track of whether the next "inline" found is
#a sentence and not a question:
get = 0

for line in lines:
    if line != "":
        l = line.split(" ")
        if 'trial' in l[0]:
            if l[1][0]=="E":
                if l[1].split('D')[1][0]=='0':
                    cond = l[1].split('I')[0][1:]
                    item = l[1].split('I')[1].split('D')[0]
                    sentences.append(cond+' '+item)
                    get = 1
                else:
                    get = 0
        elif 'inline' in l:
            if get == 1:
                sent = line.split('|')[1]
                s = len(sentences)-1
                sentences[s] = sentences[s]+' '+sent
                get = 0                

# open output text file
allsentences = open('allsentences.txt', 'w')

for row in sentences:
    allsentences.write(str(row))

allsentences.close()
!head allsentences.txt -n 4

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 678, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 322, in init_sockets
    self.stdin_port = self._bind_socket(self.stdin_socket, self.stdin_port)
  File "/opt/conda/lib/python3.10/site-packages/

1 1 As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.\n
2 1 As soon as the meeting ended, Paul gave the project files his assistant, eager to wrap up the day's tasks.\n
1 2 Unsure whether it was the right decision, Mia sold her furniture to a local dealer before moving out of her old apartment.\n
2 2 Unsure whether it was the right decision, Mia sold her furniture a local dealer before moving out of her old apartment.\n


In [2]:
#this silly bit of code formats the sentence data to look the way some later magic code will want
import csv
with open("allsentences.txt", "r") as reader:
    lineNo = 0
    conds = {1: ("PREP_DAT",True),
             2: ("PREP_DAT",False),
             3: ("COM_SBJ",True),
             4: ("COM_SBJ",False),
             5: ("FILLER", False)}
    writer = csv.DictWriter(open("script_items_pivot.csv","w"),fieldnames=["row","item","condition","ambiguity","Sentence"])
    writer.writeheader()
    for line in reader: # find the sentence, item, condition, ambiguity
        line = line.split()
        cnd = line[0]
        item = line[1]
        sentence = " ".join(line[2:])[:-2]
        condition = conds[int(cnd)][0]
        ambiguous = not conds[int(cnd)][1]
        writer.writerow({"row":lineNo,"item":item,"condition":condition,"ambiguity":ambiguous,"Sentence":sentence})
        lineNo +=1
!head script_items_pivot.csv -n 4
        

row,item,condition,ambiguity,Sentence
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks."
1,1,PREP_DAT,True,"As soon as the meeting ended, Paul gave the project files his assistant, eager to wrap up the day's tasks."
2,2,PREP_DAT,False,"Unsure whether it was the right decision, Mia sold her furniture to a local dealer before moving out of her old apartment."


Now that we have organized the experimental trials into CSV form, we can start doing analysis! First, we collect the GPT-2 surprisals with some more magic code (this will also split the sentences word by word). Let's define functions for the magic code.

In [3]:
import re

def clean(token):
    return re.sub("[^a-zA-Z0-9*.,!?\-]", "", token) # filter out non-alphanumeric or punctuation characters

def align(words, wordpieces, debug=False):
    # Remove the "not beginning of sentence" character from the wordpieces
    wordpieces = [clean(piece) for piece in wordpieces]

    aligned = [] # list containing lists of wordpieces that make up each word
    idx_word = 0 # idx of the next word
    current_pieces = [] # wordpieces that don't align with the next word

    for idx_piece, piece in enumerate(wordpieces):
        if idx_word < len(words):
            if debug: print("not EOS")
            word = words[idx_word]
        else:
            current_pieces += wordpieces[idx_piece:]
            break

        if debug: print(piece, word, piece == word[:len(piece)])

        if piece == word[:len(piece)]:
            # if the new wordpiece is aligned to the next word

            # all current pieces belong to the current word
            aligned.append(current_pieces)

            # and the new piece belongs to the next word
            idx_word += 1
            current_pieces = [piece]
        else:
            # otherwise, the new piece belongs to the current word too
            current_pieces.append(piece)

    # at EOS, all remaining wordpieces belong to the last word
    aligned.append(current_pieces)
    if debug: print("EOS, merging the rest in: " + ",".join(current_pieces))

    # First entry in aligned is always empty (first wordpiece should always match the first word)
    aligned = aligned[1:]

    # get the indices of the wordpiece that correspond to word boundaries (breaks)
    breaks = [len(pieces) for pieces in aligned]
    breaks = [0] + [sum(breaks[:i+1]) for i in range(len(breaks))]

    return aligned, breaks

In [5]:
#magic code to get GPT-2 surprisals
#source: Huang et al. 2023
!pip -q install transformers
!pip -q install torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

import csv
import re
import numpy as np

from tqdm import tqdm

# TODO option for selecting subword merges to compute

# how can we combine subwords/punctuation to get one surprisal per word?
merge_fs = {"sum_":sum}

# Load models from HF transformers
tokenizer = AutoTokenizer.from_pretrained("gpt2", add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained("gpt2")

#if args.cuda:
    #device = torch.device("cuda")
    #model.to(device)
#I commented that out for cross-platformness. SUFFER!


in_f = open("script_items_pivot.csv", "r")
stims = csv.DictReader(in_f)

out = []
for stim_row in tqdm(stims):
    inputs = tokenizer("<|endoftext|> " + stim_row["Sentence"], return_tensors="pt")
    ids = inputs["input_ids"]
    tokens = tokenizer.tokenize(stim_row["Sentence"])

    # run the model
    outputs = model(**inputs, labels=ids)
    logprobs = F.log_softmax(outputs.logits[0], 1)
    
    words = stim_row["Sentence"].split()
    piecess, breaks = align(words, tokens)
    # get surp for each word (avgd over pieces) and write it to a new row
    for i, (word, pieces) in enumerate(zip(words, piecess)):
        row = stim_row.copy() # new object, not a reference
        row["token"] = ".".join(pieces)
        row["word"] = word
        row["word_pos"] = i
        # correct for alignment difference due to initial EOS in the model input. see get_lstm.py for details
        surps = [-logprobs[j][ids[0][j+1]].item() for j in range(breaks[i], breaks[i+1])]
        surps_base2 = [surp/np.log(2.0) for surp in surps]
        for merge_fn, merge_f in merge_fs.items():
            row[merge_fn + "surprisal"] = merge_f(surps)
            #row[merge_fn + "surprisal_base2"] = merge_f(surps_base2)

        out.append(row)

with open("script_items_pivot.gpt2.csv", "w") as out_f:
    writer = csv.DictWriter(out_f, fieldnames = out[0].keys())
    writer.writeheader()
    writer.writerows(out)

!head script_items_pivot.gpt2.csv -n 4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
260it [00:56,  4.60it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


row,item,condition,ambiguity,Sentence,token,word,word_pos,sum_surprisal,sum_surprisal_base2,mean_surprisal,mean_surprisal_base2
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",As,As,0,10.346120834350586,14.926297220155575,10.346120834350586,14.926297220155575
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",soon,soon,1,4.550016403198242,6.564286100857543,4.550016403198242,6.564286100857543
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",as,as,2,0.005741414614021778,0.008283110391336642,0.005741414614021778,0.008283110391336642


Now, we can add word lengths and log frequencies

In [7]:
import csv
import math
import re

coca_freqs = csv.DictReader(open("freqs_coca.csv"))

freqs = {}
for row in coca_freqs:
    freqs[row["word"]] = int(row["count"])

def get_uni_freq(word):
    return freqs.get(re.sub("[.,?!;:']", "", word.lower()), 0)

fillers = {}
with open("script_items_pivot.gpt2.csv", "r") as rd:
    with open("words_surp_length_freq.csv","w") as wr:
        reader = csv.DictReader(rd)
        writer = csv.DictWriter(wr,fieldnames=list(reader.fieldnames)+["lg_freq","len"])
        writer.writeheader()
        for row in reader:
            fq = math.log2(get_uni_freq(row["word"])) if (re.sub("[.,?!;:]", "", row["word"].lower()) in freqs and freqs[re.sub("[.,?!;:]", "", row["word"].lower())] > 0) else "NA"
            row["lg_freq"] = fq
            row["len"] = len(row["word"])
            writer.writerow(row)
                
!head words_surp_length_freq.csv -n 4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


row,item,condition,ambiguity,Sentence,token,word,word_pos,sum_surprisal,sum_surprisal_base2,mean_surprisal,mean_surprisal_base2,lg_freq,len
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",As,As,0,10.346120834350586,14.926297220155575,10.346120834350586,14.926297220155575,19.505472374673356,2
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",soon,soon,1,4.550016403198242,6.564286100857543,4.550016403198242,6.564286100857543,14.498288270986565,4
0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",as,as,2,0.005741414614021778,0.008283110391336642,0.005741414614021778,0.008283110391336642,19.505472374673356,2


Now, we add the spillover surprisals for the last 2 words, $p_1$ and $p_2$.

In [10]:
import pandas as pd
from numpy import nan

df = pd.read_csv("words_surp_length_freq.csv")
#spillover: where the "subject" is the same and the "Sentence" but word_pos is word_pos -1 or word_pos-2
def getspillover(row,amount):
    if row["word_pos"]-amount < 0:
        return nan
    #spilloverEntries = df[(df["subject"]==row["subject"]) & (df["Sentence"]==row["Sentence"]) & (df["word_pos"] == row["word_pos"]-amount)]
    spilloverEntries = df[(df["Sentence"]==row["Sentence"]) & (df["word_pos"] == row["word_pos"]-amount)]
    return spilloverEntries["sum_surprisal"].iloc[0]

df["spillover_1"] = df.apply(lambda row: getspillover(row,1),axis=1,result_type="reduce")
print("spillover 1!")
df["spillover_2"] = df.apply(lambda row: getspillover(row,2),axis=1,result_type="reduce")
print("spillover 2")
df.to_csv("all_predictors.csv",na_rep="NA")
print("DONE")
!head all_predictors.csv -n 4

spillover 1!
spillover 2
DONE


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,row,item,condition,ambiguity,Sentence,token,word,word_pos,sum_surprisal,sum_surprisal_base2,mean_surprisal,mean_surprisal_base2,lg_freq,len,spillover_1,spillover_2
0,0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",As,As,0,10.346120834350586,14.926297220155575,10.346120834350586,14.926297220155575,19.50547237467336,2,NA,NA
1,0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",soon,soon,1,4.550016403198242,6.564286100857543,4.550016403198242,6.564286100857543,14.498288270986563,4,10.346120834350586,NA
2,0,1,PREP_DAT,False,"As soon as the meeting ended, Paul gave the project files to his assistant, eager to wrap up the day's tasks.",as,as,2,0.0057414146140217,0.0082831103913366,0.0057414146140217,0.0082831103913366,19.50547237467336,2,4.550016403198242,10.346120834350586


Now we have all of our predictor variables for each word, neatly in a single csv file.
# Time to get the eye-tracking data!
We have 2 files of interest inside `eyetracking_data`: `fp.ixs` contains first-pass times, and `ro.ixs` contains regression data. Each is a csv file outlining the item, condition, and participant, with a column for each word.

first, we need to replace all of the empty columns in `ro.ixs` and all of the 0 ms GP times in `fp.ixs` with `NA`. This will help R deal with the NA values.

In [18]:
import os

import pandas as pd
from numpy import nan



#replace 0 FP times with NA
df = pd.read_csv("eyetracking_data/fp.ixs")
df = df.astype(pd.Int64Dtype(),copy=True)
df = df.replace(0,nan)
df.to_csv("eyetracking_data/fp_clean.ixs",na_rep="NA")

#pandas should autointerepret the ,, columns of RO as NA
print(df.head())

ro_df = pd.read_csv("eyetracking_data/ro.ixs")
ro_df = ro_df.astype(pd.Int64Dtype(),copy=True)
ro_df.to_csv("eyetracking_data/ro_clean.ixs",na_rep="NA")
ro_df.head()

   seq  subj  item  cond   R1    R2    R3    R4    R5    R6  ...   R14   R15  \
0  170     1     1     5   62   343   515  <NA>   411   514  ...   258    47   
1  349     1     2     5  157   235   256  <NA>   180  <NA>  ...   352   673   
2  128     1     3     5  231  <NA>   528  <NA>   302   476  ...   269  <NA>   
3  347     1     4     5  215   388   174  <NA>   322   364  ...   546   201   
4  114     1     5     5  263  <NA>  <NA>   416  <NA>   286  ...  <NA>   358   

    R16   R17   R18   R19   R20   R21   R22   R23  
0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  
1  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  
2  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  
3  <NA>   349   321  <NA>  <NA>  <NA>  <NA>  <NA>  
4  <NA>   324   267  <NA>   341  <NA>  <NA>  <NA>  

[5 rows x 27 columns]


,seq,subj,item,cond,R1,R2,R3,R4,R5,R6,...,R14,R15,R16,R17,R18,R19,R20,R21,R22,R23
0,170,1,1,5,0,0,0,<NA>,0,0,...,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,349,1,2,5,0,0,1,<NA>,0,<NA>,...,0,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,128,1,3,5,0,<NA>,0,<NA>,0,0,...,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,347,1,4,5,0,0,1,<NA>,0,0,...,0,0,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
4,114,1,5,5,0,<NA>,<NA>,1,<NA>,0,...,<NA>,1,<NA>,1,0,<NA>,1,<NA>,<NA>,<NA>


With our eyetracking data cleaned, we can join it with the rest of our data. Because I was originally foolish and I do not want to change my old code to much, we are going to do this wide-form first, then turn it into the long-form table we want. 

In [5]:
import pandas as pd
import os
from numpy import nan
from tqdm import tqdm



df = pd.read_csv("all_predictors.csv")
pbar = tqdm(total=len(df))

i = 0
#if you are wondering why I did it this way, just know my old code was even more foolhardy
def add_eye_data_cols(row):
    global i
    i += 1
    item = row["item"]
    wordPos1 = row["word_pos"]+1
    rodf = pd.read_csv("eyetracking_data/ro_clean.ixs")
    rodf = rodf[rodf["item"]==item]
    rodf = rodf[pd.notna(rodf[f"R{wordPos1}"])]

    subj = rodf["subj"].astype(pd.Int16Dtype(),copy=True).apply(lambda val:f"ro_{val}")
    row = pd.concat([row,pd.Series(list(rodf[f"R{wordPos1}"]),list(subj),dtype=pd.Int16Dtype())])
    
    fpdf = pd.read_csv("eyetracking_data/fp_clean.ixs")
    fpdf = fpdf[fpdf["item"]==item]
    fpdf = fpdf[pd.notna(fpdf[f"R{wordPos1}"])]

    subj = fpdf["subj"].astype(pd.Int16Dtype(),copy=True).apply(lambda val:f"fp_{val}")
    row = pd.concat([row,pd.Series(list(fpdf[f"R{wordPos1}"]),list(subj),dtype=pd.Int16Dtype())])
    if i % 100 == 0 and i > 0:
        pbar.update(100)
    return row


df = df.apply(add_eye_data_cols,1)
#for subj in range(1,77):
    #df = df.astype({f"ro_{subj}":pd.Int32Dtype(),f"fp_{subj}":pd.Int32Dtype()})

#0-4 fakerow,Sentence,Unnamed: 0, ambiguity, condition
fpidxs = list(range(4,81))
#82-86 item,len,lg_freq,mean_surp,meansurp2
roidxs = list(range(86,163))
#164-171 row,spillover1,spillover2,sumsurp,sumsurp2,token,word,pos
df = df.iloc[:,[163,3,81,2,0,169,170,166,164,165,82,83]+fpidxs+roidxs]


df.to_csv("wideform_data.csv",na_rep="NA")
print("DONE")
    #takes a series row, gets eyetracking data, adds shit to it


 98%|█████████▊| 3900/3973 [05:04<00:05, 12.81it/s]

 98%|█████████▊| 3900/3973 [02:02<00:02, 30.96it/s]

DONE


Now, we will use the magic of R to turn this mess into a longform table as god intended

In [ ]:
!conda update r-rlang -y

In [8]:
install.packages("rlang")
install.packages("tidyr", repos = "https://cloud.r-project.org")
install.packages("dplyr", repos = "https://cloud.r-project.org")
library("tidyr")
library("dplyr")
orig_data <- read.csv("wideform_data.csv")
fp_data <- orig_data %>% pivot_longer(cols=14:90,names_to="subject",values_to="fp")#0 idx?
fp_data[14:90]<-NULL
#fp_data[18:21]<-NULL
ro_data <- orig_data %>% pivot_longer(cols=91:167,names_to="subject",values_to="ro")#0 idx?
ro_data[14:90]<-NULL
#ro_data[18:21]<-NULL
ro_data$subject<-gsub("ro","fp",ro_data$subject)
head(fp_data$subject)
head(ro_data$subject)


merged_data <- right_join(fp_data,ro_data)
write.csv(merged_data,"data.pivot.csv")

trying URL 'https://cran.r-project.org/src/contrib/rlang_1.1.6.tar.gz'
Content type 'application/x-gzip' length 767928 bytes (749 KB)
downloaded 749 KB

* installing *source* package ‘rlang’ ...
** package ‘rlang’ successfully unpacked and MD5 sums checked
** using staged installation
** libs


x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG -I./rlang/  -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib  -fvisibility=hidden -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1671440533574/work=/usr/local/src/conda/r-base-4.2.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c capture.c -o capture.o
x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG -I./rlang/  -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib  -fvisibility=hidden -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-b

installing to /opt/conda/lib/R/library/00LOCK-rlang/00new/rlang/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

** testing if installed package keeps a record of temporary installation path
* DONE (rlang)

The downloaded source packages are in
	‘/tmp/RtmpqFOWQ4/downloaded_packages’
trying URL 'https://cloud.r-project.org/src/contrib/tidyr_1.3.1.tar.gz'
Content type 'application/x-gzip' length 809058 bytes (790 KB)
downloaded 790 KB

* installing *source* package ‘tidyr’ ...
** package ‘tidyr’ successfully unpacked and MD5 sums checked
** using staged installation
** libs


x86_64-conda-linux-gnu-c++ -std=gnu++14 -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/cpp11/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -fvisibility-inlines-hidden -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1671440533574/work=/usr/local/src/conda/r-base-4.2.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c cpp11.cpp -o cpp11.o
x86_64-conda-linux-gnu-c++ -std=gnu++14 -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/cpp11/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -fvisibility-inlines-hidden -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-pro

installing to /opt/conda/lib/R/library/00LOCK-tidyr/00new/tidyr/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

** testing if installed package keeps a record of temporary installation path
* DONE (tidyr)

The downloaded source packages are in
	‘/tmp/RtmpqFOWQ4/downloaded_packages’
trying URL 'https://cloud.r-project.org/src/contrib/dplyr_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1207521 bytes (1.2 MB)
downloaded 1.2 MB

* installing *source* package ‘dplyr’ ...
** package ‘dplyr’ successfully unpacked and MD5 sums checked


x86_64-conda-linux-gnu-c++ -std=gnu++14 -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -fvisibility-inlines-hidden -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1671440533574/work=/usr/local/src/conda/r-base-4.2.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c chop.cpp -o chop.o
x86_64-conda-linux-gnu-c++ -std=gnu++14 -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -fvisibility-inlines-hidden -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdeb

installing to /opt/conda/lib/R/library/00LOCK-dplyr/00new/dplyr/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

** testing if installed package keeps a record of temporary installation path
* DONE (dplyr)

The downloaded source packages are in
	‘/tmp/RtmpqFOWQ4/downloaded_packages’


[1] "fp_1"  "fp_10" "fp_11" "fp_12" "fp_13" "fp_14"

[1] "fp_1"  "fp_10" "fp_11" "fp_12" "fp_13" "fp_14"

Joining with `by = join_by(X, row, condition, item, ambiguity, Sentence, word,
word_pos, sum_surprisal, spillover_1, spillover_2, len, lg_freq, subject)`


In [11]:
install.packages("lme4")
filler_data = read.csv("data.pivot.csv")
filler_data$subject <- as.factor(filler_data$subject)
filler.model_surp = lmer(data=filler_data,
                           fp ~ sum_surprisal+spillover_1+spillover_2+word_pos+lg_freq*len+(1 | subject),
control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

also installing the dependency ‘nloptr’


trying URL 'https://cran.r-project.org/src/contrib/nloptr_2.2.1.tar.gz'
Content type 'application/x-gzip' length 2253853 bytes (2.1 MB)
downloaded 2.1 MB

trying URL 'https://cran.r-project.org/src/contrib/lme4_1.1-37.tar.gz'
Content type 'application/x-gzip' length 3313529 bytes (3.2 MB)
downloaded 3.2 MB

* installing *source* package ‘nloptr’ ...
** package ‘nloptr’ successfully unpacked and MD5 sums checked
** using staged installation


checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C++... yes
checking whether x86_64-conda-linux-gnu-c++ -std=gnu++14 accepts -g... yes
checking for x86_64-conda-linux-gnu-c++ -std=gnu++14 option to enable C++11 features... none needed
checking how to run the C++ preprocessor... x86_64-conda-linux-gnu-c++ -std=gnu++14 -E
checking whether the compiler supports GNU C++... (cached) yes
checking whether x86_64-conda-linux-gnu-c++ -std=gnu++14 accepts -g... (cached) yes
checking for x86_64-conda-linux-gnu-c++ -std=gnu++14 option to enable C++11 features... (cached) none needed
checking for pkg-config... no
checking for cmake... no

------------------ CMAKE NOT FOUND --------------------

CMake was not found on the PATH. Please install CMake:

 - sudo yum install cmake 

Warning message in install.packages("lme4"):
“installation of package ‘nloptr’ had non-zero exit status”
ERROR: configuration failed for package ‘nloptr’
* removing ‘/opt/conda/lib/R/library/nloptr’
Warning message in install.packages("lme4"):
“installation of package ‘lme4’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

ERROR: dependency ‘nloptr’ is not available for package ‘lme4’
* removing ‘/opt/conda/lib/R/library/lme4’

The downloaded source packages are in
	‘/tmp/RtmpqFOWQ4/downloaded_packages’


ERROR: Error in lmer(data = filler_data, fp ~ sum_surprisal + spillover_1 + spillover_2 + : could not find function "lmer"


In [ ]:
import os
os.system("apt install cmake -y")